In [151]:
import pandas as pd
import numpy as np 
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from catboost import CatBoostRegressor,CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
import math
from rdkit.Chem import Descriptors
from rdkit import Chem
from mordred import Calculator,descriptors
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV,RepeatedStratifiedKFold,train_test_split,StratifiedKFold,KFold,cross_val_score
from sklearn.feature_selection import RFE,VarianceThreshold,RFECV,SelectFromModel
from sklearn import metrics

In [152]:
#Integrating cell morphology with gene expression and chemical structure to aid mitochondrial toxicity detection
#qHTS assay for small molecule disruptors of the mitochondrial membrane potential
#2,000 HepG2 cells in 5 uL/well
#qHTS assay for small molecule disruptors of the mitochondrial membrane potential - cell viability
#2,000 HepG2 cells in 5 uL/well were dispensed into black,CellTiter-Glo reagent was added at 4 uL/well and incubated 30 min at room temperature
#AID 720637
#AID 720635 and AID 720634
#測ATP量以及染色比例取交集
df1 = pd.read_csv("data/cp_pre")
d3 = pd.DataFrame()
d3["Inchi"] = df1["Standard_name"]
d3["Toxicity"] = df1["PUBCHEM_ACTIVITY_OUTCOME"]
d3.reset_index(inplace=True)
d3.drop("index",axis = 1,inplace = True)
for i in range(len(d3)):
    if d3["Inchi"][i] == None:
        print(i)
d3["Toxicity"].value_counts()

Toxicity
0    404
1     92
Name: count, dtype: int64

In [153]:
#Integrating cell morphology with gene expression and chemical structure to aid mitochondrial toxicity detection
#qHTS assay for small molecule disruptors of the mitochondrial membrane potential
#2,000 HepG2 cells in 5 uL/well
#qHTS assay for small molecule disruptors of the mitochondrial membrane potential - cell viability
#2,000 HepG2 cells in 5 uL/well were dispensed into black,CellTiter-Glo reagent was added at 4 uL/well and incubated 30 min at room temperature
#AID 720637
#AID 720635 and AID 720634
#測ATP量以及染色比例取交集
df2 = pd.read_csv("data/go_pre")
d5 = pd.DataFrame()
d5["Inchi"] = df2["Standard_name"]
d5["Toxicity"] = df2["PUBCHEM_ACTIVITY_OUTCOME"]
d5.reset_index(inplace=True)
d5.drop("index",axis = 1,inplace = True)
for i in range(len(d5)):
    if d5["Inchi"][i] == None:
        print(i)
d5["Toxicity"].value_counts()

Toxicity
0    607
1    161
Name: count, dtype: int64

In [154]:
#Using Machine Learning Methods and Structural Alerts for Prediction of Mitochondrial Toxicity
#Training_dataset_31072019.sdf
#AID 720637
#should be 5761
#Zhang, ChemBl, Tox21,PubChem
data = pd.read_csv("data/toxicity2.csv",index_col=0)
data = data.replace({"Activity" :{"Inactive" : 0,"Active" : 1,"Inactive, Active" : -1}})
d1 = pd.DataFrame()
d1["Inchi"] = data["InChICode_standardised"]
d1["Toxicity"] = data["Activity"]
d1.reset_index(inplace=True)
d1.drop("index",axis = 1,inplace = True)
for i in range(len(d1)):
    if d1["Inchi"][i] == None:
        print(i)
d1["Toxicity"].value_counts()

Toxicity
 0    4931
 1     824
-1       1
Name: count, dtype: int64

In [155]:
#Integrating cell morphology with gene expression and chemical structure to aid mitochondrial toxicity detection
#data is from Using Machine Learning Methods and Structural Alerts for Prediction of Mitochondrial Toxicity
#Zhang
data1 = pd.read_csv("data/New_test_244.csv",index_col=0)
d2 = pd.DataFrame()
data1.reset_index(inplace=True)
d2["Inchi"] = data1["InChICode_standardised"]
d2["Toxicity"] = data1["Activity"].astype(int)
d2.reset_index(inplace=True)
d2.drop("index",axis = 1,inplace = True)
for i in range(len(d2)):
    if d2["Inchi"][i] == None:
        print(i)
d2["Toxicity"].value_counts()

Toxicity
0    197
1     47
Name: count, dtype: int64

In [156]:
#In silico prediction of mitochondrial toxicity of chemicals using machine learning methods
#PubChem bioassay database (AID 720635, AID 1347389, AID 720637), 
#AID 1347389:qHTS assay for small molecule disruptors of mitochondrial membrane potential screened against the NCATS CANVASS Library
#2,000 HepG2 cells in 5 uL/well 
#DrugBank (Wishart et al., 2018), and literature (Zhang et al., 2009)
d4 = pd.read_csv("data/silico_pred.csv",index_col=0)
d4["Toxicity"] = d4["Toxicity"].astype(int)
d4.reset_index(inplace=True)
d4.drop("index",axis = 1,inplace = True)
for i in range(len(d4)):
    if d4["Inchi"][i] == None:
        print(i)
d4["Toxicity"].value_counts()

Toxicity
0    1869
1    1537
Name: count, dtype: int64

In [157]:
#Using Machine Learning Methods and Structural Alerts for Prediction of Mitochondrial Toxicity
#Tox21 SR-MMP” (stress response – mitochondrial membrane potential
#From https://tripod.nih.gov/tox21/challenge/data.jsp
suppl = Chem.SDMolSupplier('data/test_dataset_Tox21.sdf')
mol_list = []
inchi_list = []
tox_list = []
d6 = pd.DataFrame()
# 遍歷SDF文件中的化合物
for mol in suppl:
    if mol is None:
        continue

    # 提取InChI
    inchi_list.append(Chem.MolToInchi(mol))
    tox_list.append(mol.GetProp("SR-MMP"))  # 假設毒性值存儲在名為"toxicity"的屬性中
    mol_list.append(mol)
    # 將InChI和毒性值添加到DataFrame
d6["Mol"] = mol_list
d6["Inchi"] = inchi_list
d6["Toxicity"] = np.array(tox_list).astype(float).astype(int)
d6.reset_index(inplace=True)
d6.drop("index",axis = 1,inplace = True)
d6["Toxicity"].value_counts()

[12:01:34] WARNING: Omitted undefined stereo

[12:01:34] WARNING: Charges were rearranged; Proton(s) added/removed

[12:01:34] WARNING: Charges were rearranged; Omitted undefined stereo

[12:01:34] WARNING: Omitted undefined stereo

[12:01:34] WARNING: Omitted undefined stereo

[12:01:34] WARNING: Omitted undefined stereo

[12:01:34] WARNING: Omitted undefined stereo

[12:01:34] WARNING: Omitted undefined stereo

[12:01:34] WARNING: Omitted undefined stereo

[12:01:34] WARNING: Omitted undefined stereo

[12:01:34] WARNING: Omitted undefined stereo

[12:01:34] WARNING: Omitted undefined stereo

[12:01:34] WARNING: Omitted undefined stereo

[12:01:34] WARNING: Omitted undefined stereo

[12:01:34] WARNING: Charges were rearranged

[12:01:34] WARNING: Omitted undefined stereo

[12:01:34] WARNING: Omitted undefined stereo

[12:01:34] WARNING: Omitted undefined stereo

[12:01:34] WARNING: Omitted undefined stereo

[12:01:34] WARNING: Salt was disconnected; Proton(s) added/removed

[12:01:34]

Toxicity
0    614
1     92
Name: count, dtype: int64

In [158]:
#combine d1 and d4 into d1(same assay)
#delete the overlap part
num = []
counter1 = []
counter2 = []
for i in range(len(d1)):
    for y in range(len(d4)):
        if(d1["Inchi"][i] == d4["Inchi"][y]):
            num.append(y)
            print(i,":",d1["Inchi"][i]," // ",y,":",d4["Inchi"][y])
            print(i,":",d1["Toxicity"][i]," // ",y,":",d4["Toxicity"][y])
            if d1["Toxicity"][i] != d4["Toxicity"][y]:
                counter1.append(i)
                counter2.append(y)

5 : InChI=1S/C11H17BrN/c1-4-13(2,3)9-10-7-5-6-8-11(10)12/h5-8H,4,9H2,1-3H3/q+1  //  1286 : InChI=1S/C11H17BrN/c1-4-13(2,3)9-10-7-5-6-8-11(10)12/h5-8H,4,9H2,1-3H3/q+1
5 : 0  //  1286 : 0
7 : InChI=1S/C21H29FO5/c1-18-7-5-13(24)9-12(18)3-4-15-14-6-8-20(27,17(26)11-23)19(14,2)10-16(25)21(15,18)22/h9,14-16,23,25,27H,3-8,10-11H2,1-2H3/t14-,15-,16-,18-,19-,20-,21-/m0/s1  //  1753 : InChI=1S/C21H29FO5/c1-18-7-5-13(24)9-12(18)3-4-15-14-6-8-20(27,17(26)11-23)19(14,2)10-16(25)21(15,18)22/h9,14-16,23,25,27H,3-8,10-11H2,1-2H3/t14-,15-,16-,18-,19-,20-,21-/m0/s1
7 : 0  //  1753 : 0
9 : InChI=1S/C23H27N/c1-23(2,3)21-14-12-18(13-15-21)16-24(4)17-20-10-7-9-19-8-5-6-11-22(19)20/h5-15H,16-17H2,1-4H3  //  913 : InChI=1S/C23H27N/c1-23(2,3)21-14-12-18(13-15-21)16-24(4)17-20-10-7-9-19-8-5-6-11-22(19)20/h5-15H,16-17H2,1-4H3
9 : 0  //  913 : 0
14 : InChI=1S/C10H22N4/c11-10(12)13-6-9-14-7-4-2-1-3-5-8-14/h1-9H2,(H4,11,12,13)  //  1538 : InChI=1S/C10H22N4/c11-10(12)13-6-9-14-7-4-2-1-3-5-8-14/h1-9H2,(H4,11,12,13)
1

121 : InChI=1S/C19H28NO3/c1-20(2)13-12-17(14-20)23-18(21)19(22,16-10-6-7-11-16)15-8-4-3-5-9-15/h3-5,8-9,16-17,22H,6-7,10-14H2,1-2H3/q+1  //  1551 : InChI=1S/C19H28NO3/c1-20(2)13-12-17(14-20)23-18(21)19(22,16-10-6-7-11-16)15-8-4-3-5-9-15/h3-5,8-9,16-17,22H,6-7,10-14H2,1-2H3/q+1
121 : 0  //  1551 : 0
125 : InChI=1S/C21H13F3N2O4/c22-21(23,24)12-5-3-6-13(11-12)26-17-16(9-4-10-25-17)19(28)30-20-15-8-2-1-7-14(15)18(27)29-20/h1-11,20H,(H,25,26)  //  2703 : InChI=1S/C21H13F3N2O4/c22-21(23,24)12-5-3-6-13(11-12)26-17-16(9-4-10-25-17)19(28)30-20-15-8-2-1-7-14(15)18(27)29-20/h1-11,20H,(H,25,26)
125 : 1  //  2703 : 1
128 : InChI=1S/C12H8Cl2O2S/c13-7-1-3-9(15)11(5-7)17-12-6-8(14)2-4-10(12)16/h1-6,15-16H  //  1903 : InChI=1S/C12H8Cl2O2S/c13-7-1-3-9(15)11(5-7)17-12-6-8(14)2-4-10(12)16/h1-6,15-16H
128 : 1  //  1903 : 1
130 : InChI=1S/C8H11N7S/c9-7(10)14-12-5-1-3-6(4-2-5)13-15-8(11)16/h1-4,12H,(H2,11,16)(H4,9,10,14)  //  2423 : InChI=1S/C8H11N7S/c9-7(10)14-12-5-1-3-6(4-2-5)13-15-8(11)16/h1-4,12H,(H2,11,

206 : InChI=1S/C17H16Cl2O3/c1-11(2)22-16(20)17(21,12-3-7-14(18)8-4-12)13-5-9-15(19)10-6-13/h3-11,21H,1-2H3  //  2612 : InChI=1S/C17H16Cl2O3/c1-11(2)22-16(20)17(21,12-3-7-14(18)8-4-12)13-5-9-15(19)10-6-13/h3-11,21H,1-2H3
206 : 1  //  2612 : 1
208 : InChI=1S/C7H10N2/c1-5-3-2-4-6(8)7(5)9/h2-4H,8-9H2,1H3  //  3376 : InChI=1S/C7H10N2/c1-5-3-2-4-6(8)7(5)9/h2-4H,8-9H2,1H3
208 : 1  //  3376 : 1
209 : InChI=1S/C14H30N2O4/c1-15(2,3)9-11-19-13(17)7-8-14(18)20-12-10-16(4,5)6/h7-12H2,1-6H3/q+2  //  28 : InChI=1S/C14H30N2O4/c1-15(2,3)9-11-19-13(17)7-8-14(18)20-12-10-16(4,5)6/h7-12H2,1-6H3/q+2
209 : 0  //  28 : 0
214 : InChI=1S/C4H9NO3/c1-2(6)3(5)4(7)8/h2-3,6H,5H2,1H3,(H,7,8)/t2-,3+/m1/s1  //  15 : InChI=1S/C4H9NO3/c1-2(6)3(5)4(7)8/h2-3,6H,5H2,1H3,(H,7,8)/t2-,3+/m1/s1
214 : 0  //  15 : 0
221 : InChI=1S/C8H17NO2/c1-6(2)3-7(5-9)4-8(10)11/h6-7H,3-5,9H2,1-2H3,(H,10,11)/t7-/m0/s1  //  1775 : InChI=1S/C8H17NO2/c1-6(2)3-7(5-9)4-8(10)11/h6-7H,3-5,9H2,1-2H3,(H,10,11)/t7-/m0/s1
221 : 0  //  1775 : 0
222 : InCh

307 : InChI=1S/C16H10O/c17-14-9-7-12-5-4-10-2-1-3-11-6-8-13(14)16(12)15(10)11/h1-9,17H  //  2158 : InChI=1S/C16H10O/c17-14-9-7-12-5-4-10-2-1-3-11-6-8-13(14)16(12)15(10)11/h1-9,17H
307 : 1  //  2158 : 1
309 : InChI=1S/C10H6Cl8/c11-3-1-2-4(5(3)12)9(16)7(14)6(13)8(2,15)10(9,17)18/h2-5H,1H2  //  2508 : InChI=1S/C10H6Cl8/c11-3-1-2-4(5(3)12)9(16)7(14)6(13)8(2,15)10(9,17)18/h2-5H,1H2
309 : 0  //  2508 : 1
311 : InChI=1S/C6H4Cl2N2O2/c7-4-1-3(10(11)12)2-5(8)6(4)9/h1-2H,9H2  //  1946 : InChI=1S/C6H4Cl2N2O2/c7-4-1-3(10(11)12)2-5(8)6(4)9/h1-2H,9H2
311 : 1  //  1946 : 1
317 : InChI=1S/C21H24O2/c1-3-20-11-9-17-16-8-6-15(22)13-14(16)5-7-18(17)19(20)10-12-21(20,23)4-2/h2,9,11,13,18-19,23H,3,5-8,10,12H2,1H3/t18-,19+,20+,21+/m1/s1  //  1683 : InChI=1S/C21H24O2/c1-3-20-11-9-17-16-8-6-15(22)13-14(16)5-7-18(17)19(20)10-12-21(20,23)4-2/h2,9,11,13,18-19,23H,3,5-8,10,12H2,1H3/t18-,19+,20+,21+/m1/s1
317 : 0  //  1683 : 0
320 : InChI=1S/C12H16O7/c13-5-8-9(15)10(16)11(17)12(19-8)18-7-3-1-6(14)2-4-7/h1-4,8-17H,5H

414 : InChI=1S/C6H5NO3/c8-6-3-1-5(2-4-6)7(9)10/h1-4,8H  //  3047 : InChI=1S/C6H5NO3/c8-6-3-1-5(2-4-6)7(9)10/h1-4,8H
414 : 1  //  3047 : 1
420 : InChI=1S/C12H21N/c1-10-3-9-4-11(2,6-10)8-12(13,5-9)7-10/h9H,3-8,13H2,1-2H3  //  1510 : InChI=1S/C12H21N/c1-10-3-9-4-11(2,6-10)8-12(13,5-9)7-10/h9H,3-8,13H2,1-2H3
420 : 0  //  1510 : 0
421 : InChI=1S/C11H16N2O/c1-7-5-4-6-8(2)10(7)13-11(14)9(3)12/h4-6,9H,12H2,1-3H3,(H,13,14)  //  228 : InChI=1S/C11H16N2O/c1-7-5-4-6-8(2)10(7)13-11(14)9(3)12/h4-6,9H,12H2,1-3H3,(H,13,14)
421 : 0  //  228 : 0
435 : InChI=1S/C16H28O/c1-10-14-8-12(16(10,2)3)9-15(14)11-5-4-6-13(17)7-11/h10-15,17H,4-9H2,1-3H3  //  2355 : InChI=1S/C16H28O/c1-10-14-8-12(16(10,2)3)9-15(14)11-5-4-6-13(17)7-11/h10-15,17H,4-9H2,1-3H3
435 : 1  //  2355 : 1
439 : InChI=1S/C6H7ClN2/c7-4-1-2-5(8)6(9)3-4/h1-3H,8-9H2  //  2981 : InChI=1S/C6H7ClN2/c7-4-1-2-5(8)6(9)3-4/h1-3H,8-9H2
439 : 1  //  2981 : 1
441 : InChI=1S/C14H16ClO5PS/c1-4-17-21(22,18-5-2)20-10-6-7-11-9(3)13(15)14(16)19-12(11)8-10/h6-8H,4-

...

5139 : InChI=1S/C2H7NO3S/c3-1-2-7(4,5)6/h1-3H2,(H,4,5,6)  //  2293 : InChI=1S/C2H7NO3S/c3-1-2-7(4,5)6/h1-3H2,(H,4,5,6)
5139 : 0  //  2293 : 1
5140 : InChI=1S/C21H26O5/c1-19-7-5-13(23)9-12(19)3-4-14-15-6-8-21(26,17(25)11-22)20(15,2)10-16(24)18(14)19/h5,7,9,14-15,18,22,26H,3-4,6,8,10-11H2,1-2H3/t14-,15-,18+,19-,20-,21-/m0/s1  //  1889 : InChI=1S/C21H26O5/c1-19-7-5-13(23)9-12(19)3-4-14-15-6-8-21(26,17(25)11-22)20(15,2)10-16(24)18(14)19/h5,7,9,14-15,18,22,26H,3-4,6,8,10-11H2,1-2H3/t14-,15-,18+,19-,20-,21-/m0/s1
5140 : 0  //  1889 : 1
5141 : InChI=1S/C14H9NO2/c15-8-5-6-11-12(7-8)14(17)10-4-2-1-3-9(10)13(11)16/h1-7H,15H2  //  1926 : InChI=1S/C14H9NO2/c15-8-5-6-11-12(7-8)14(17)10-4-2-1-3-9(10)13(11)16/h1-7H,15H2
5141 : 1  //  1926 : 1
5149 : InChI=1S/C13H13N3O/c14-10-3-1-9(2-4-10)13(17)16-12-7-5-11(15)6-8-12/h1-8H,14-15H2,(H,16,17)  //  1970 : InChI=1S/C13H13N3O/c14-10-3-1-9(2-4-10)13(17)16-12-7-5-11(15)6-8-12/h1-8H,14-15H2,(H,16,17)
5149 : 1  //  1970 : 1
5150 : InChI=1S/C24H28N2O5/c1-2-31-2

5258 : InChI=1S/C4H11N5/c1-9(2)4(7)8-3(5)6/h1-2H3,(H5,5,6,7,8)  //  1877 : InChI=1S/C4H11N5/c1-9(2)4(7)8-3(5)6/h1-2H3,(H5,5,6,7,8)
5258 : 0  //  1877 : 1
5258 : InChI=1S/C4H11N5/c1-9(2)4(7)8-3(5)6/h1-2H3,(H5,5,6,7,8)  //  2938 : InChI=1S/C4H11N5/c1-9(2)4(7)8-3(5)6/h1-2H3,(H5,5,6,7,8)
5258 : 0  //  2938 : 1
5260 : InChI=1S/C18H15O4P/c19-23(20-16-10-4-1-5-11-16,21-17-12-6-2-7-13-17)22-18-14-8-3-9-15-18/h1-15H  //  3256 : InChI=1S/C18H15O4P/c19-23(20-16-10-4-1-5-11-16,21-17-12-6-2-7-13-17)22-18-14-8-3-9-15-18/h1-15H
5260 : 1  //  3256 : 1
5268 : InChI=1S/C18H19N3O3S/c1-21(16-4-2-3-9-19-16)10-11-24-14-7-5-13(6-8-14)12-15-17(22)20-18(23)25-15/h2-9,15H,10-12H2,1H3,(H,20,22,23)  //  2031 : InChI=1S/C18H19N3O3S/c1-21(16-4-2-3-9-19-16)10-11-24-14-7-5-13(6-8-14)12-15-17(22)20-18(23)25-15/h2-9,15H,10-12H2,1H3,(H,20,22,23)
5268 : 1  //  2031 : 1
5271 : InChI=1S/C12H8Br2O/c13-9-1-5-11(6-2-9)15-12-7-3-10(14)4-8-12/h1-8H  //  2622 : InChI=1S/C12H8Br2O/c13-9-1-5-11(6-2-9)15-12-7-3-10(14)4-8-12/h1-8H
5

5393 : InChI=1S/C15H18I3NO3/c1-3-5-12(20)19-14-11(17)7-10(16)9(13(14)18)6-8(4-2)15(21)22/h7-8H,3-6H2,1-2H3,(H,19,20)(H,21,22)  //  1076 : InChI=1S/C15H18I3NO3/c1-3-5-12(20)19-14-11(17)7-10(16)9(13(14)18)6-8(4-2)15(21)22/h7-8H,3-6H2,1-2H3,(H,19,20)(H,21,22)
5393 : 0  //  1076 : 0
5395 : InChI=1S/C20H21ClO4/c1-13(2)24-19(23)20(3,4)25-17-11-7-15(8-12-17)18(22)14-5-9-16(21)10-6-14/h5-13H,1-4H3  //  3036 : InChI=1S/C20H21ClO4/c1-13(2)24-19(23)20(3,4)25-17-11-7-15(8-12-17)18(22)14-5-9-16(21)10-6-14/h5-13H,1-4H3
5395 : 0  //  3036 : 1
5396 : InChI=1S/C17H20O2/c1-11-9-13(5-7-15(11)18)17(3,4)14-6-8-16(19)12(2)10-14/h5-10,18-19H,1-4H3  //  2142 : InChI=1S/C17H20O2/c1-11-9-13(5-7-15(11)18)17(3,4)14-6-8-16(19)12(2)10-14/h5-10,18-19H,1-4H3
5396 : 1  //  2142 : 1
5399 : InChI=1S/C7H7NO2/c1-10-7(9)6-3-2-4-8-5-6/h2-5H,1H3  //  1022 : InChI=1S/C7H7NO2/c1-10-7(9)6-3-2-4-8-5-6/h2-5H,1H3
5399 : 0  //  1022 : 0
5403 : InChI=1S/C14H10/c1-3-7-13-11(5-1)9-10-12-6-2-4-8-14(12)13/h1-10H  //  3345 : InChI=1S/C14

5530 : InChI=1S/C16H17N3O4S/c1-8-7-24-15-11(14(21)19(15)12(8)16(22)23)18-13(20)10(17)9-5-3-2-4-6-9/h2-6,10-11,15H,7,17H2,1H3,(H,18,20)(H,22,23)/t10-,11-,15-/m1/s1  //  1172 : InChI=1S/C16H17N3O4S/c1-8-7-24-15-11(14(21)19(15)12(8)16(22)23)18-13(20)10(17)9-5-3-2-4-6-9/h2-6,10-11,15H,7,17H2,1H3,(H,18,20)(H,22,23)/t10-,11-,15-/m1/s1
5530 : 0  //  1172 : 0
5531 : InChI=1S/C6H7NO3S/c7-5-2-1-3-6(4-5)11(8,9)10/h1-4H,7H2,(H,8,9,10)  //  2629 : InChI=1S/C6H7NO3S/c7-5-2-1-3-6(4-5)11(8,9)10/h1-4H,7H2,(H,8,9,10)
5531 : 0  //  2629 : 1
5533 : InChI=1S/C31H52O3/c1-21(2)13-10-14-22(3)15-11-16-23(4)17-12-19-31(9)20-18-28-26(7)29(33-27(8)32)24(5)25(6)30(28)34-31/h21-23H,10-20H2,1-9H3/t22-,23-,31-/m1/s1  //  1745 : InChI=1S/C31H52O3/c1-21(2)13-10-14-22(3)15-11-16-23(4)17-12-19-31(9)20-18-28-26(7)29(33-27(8)32)24(5)25(6)30(28)34-31/h21-23H,10-20H2,1-9H3/t22-,23-,31-/m1/s1
5533 : 0  //  1745 : 0
5537 : InChI=1S/C6H5ClN2O3/c7-3-1-4(8)6(10)2-5(3)9(11)12/h1-2,10H,8H2  //  3359 : InChI=1S/C6H5ClN2O3/c7-3-1-4(8

5623 : InChI=1S/C8H5N3O3S2/c12-7(5-2-1-3-15-5)10-8-9-4-6(16-8)11(13)14/h1-4H,(H,9,10,12)  //  1915 : InChI=1S/C8H5N3O3S2/c12-7(5-2-1-3-15-5)10-8-9-4-6(16-8)11(13)14/h1-4H,(H,9,10,12)
5623 : 1  //  1915 : 1
5624 : InChI=1S/C11H6Br3NOS/c12-6-1-3-7(4-2-6)15-11(16)9-5-8(13)10(14)17-9/h1-5H,(H,15,16)  //  2504 : InChI=1S/C11H6Br3NOS/c12-6-1-3-7(4-2-6)15-11(16)9-5-8(13)10(14)17-9/h1-5H,(H,15,16)
5624 : 1  //  2504 : 1
5637 : InChI=1S/C7H9NO/c1-5-2-3-7(9)6(8)4-5/h2-4,9H,8H2,1H3  //  2357 : InChI=1S/C7H9NO/c1-5-2-3-7(9)6(8)4-5/h2-4,9H,8H2,1H3
5637 : 1  //  2357 : 1
5642 : InChI=1S/C2H6O3S2/c3-7(4,5)2-1-6/h6H,1-2H2,(H,3,4,5)  //  531 : InChI=1S/C2H6O3S2/c3-7(4,5)2-1-6/h6H,1-2H2,(H,3,4,5)
5642 : 0  //  531 : 0
5644 : InChI=1S/C12H15N/c1-9-8-12(2,3)13-11-7-5-4-6-10(9)11/h4-8,13H,1-3H3  //  3337 : InChI=1S/C12H15N/c1-9-8-12(2,3)13-11-7-5-4-6-10(9)11/h4-8,13H,1-3H3
5644 : 0  //  3337 : 1
5646 : InChI=1S/C12H16O3/c1-2-3-4-9-15-12(14)10-5-7-11(13)8-6-10/h5-8,13H,2-4,9H2,1H3  //  2266 : InChI=1S/C12H1

In [159]:
len(num),len(counter1),len(counter2)

(1592, 279, 279)

In [160]:
d4_ = d4.drop(num)
d4_.reset_index(inplace=True)
d4_.drop("index",axis = 1,inplace = True)
d4_.drop("Smile",axis = 1,inplace = True)
d4_["Toxicity"].value_counts()

Toxicity
0    1106
1     708
Name: count, dtype: int64

In [161]:
#Have overlap index, so weird....
#Some conpound's mol will be None when we 
counter1 = set(counter1)
d1_ = d1.drop(counter1)
d1_.reset_index(inplace=True)
d1_.drop("index",axis = 1,inplace = True)
d1_["Mol"] = [Chem.MolFromInchi(x) for x in d1_["Inchi"]]
d1_["Toxicity"].value_counts()

[12:04:02] WARNING: not removing hydrogen atom without neighbors
[12:04:02] Explicit valence for atom # 0 Cl, 5, is greater than permitted
[12:04:02] ERROR: Explicit valence for atom # 0 Cl, 5, is greater than permitted

[12:04:03] WARNING:  Problems/mismatches: Mobile-H( Hydrogens: Locations or number, Number; Charge(s): Do not match; Proton balance: Does not match)



Toxicity
0    4688
1     792
Name: count, dtype: int64

In [162]:
d_total = pd.concat([d1_,d4_],axis = 0)
d_total.reset_index(inplace=True)
d_total.drop("index",axis = 1,inplace = True)
d_total["Toxicity"].value_counts()

Toxicity
0    5794
1    1500
Name: count, dtype: int64

In [163]:
#combine d_total and cp and ge into dtotal(same assay)
#delete the overlap part
num = []
counter1 = []
counter2 = []
for i in range(len(d_total)):
    for y in range(len(d3)):
        if(d_total["Inchi"][i] == d3["Inchi"][y]):
            num.append(y)
            print(i,":",d_total["Inchi"][i]," // ",y,":",d3["Inchi"][y])
            print(i,":",d_total["Toxicity"][i]," // ",y,":",d3["Toxicity"][y])
            if d_total["Toxicity"][i] != d3["Toxicity"][y]:
                counter1.append(i)
                counter2.append(y)

0 : InChI=1S/C10H19N5S/c1-6(2)11-8-13-9(12-7(3)4)15-10(14-8)16-5/h6-7H,1-5H3,(H2,11,12,13,14,15)  //  32 : InChI=1S/C10H19N5S/c1-6(2)11-8-13-9(12-7(3)4)15-10(14-8)16-5/h6-7H,1-5H3,(H2,11,12,13,14,15)
0 : 0  //  32 : 0
4 : InChI=1S/C16H13ClN2O/c1-19-14-8-7-12(17)9-13(14)16(18-10-15(19)20)11-5-3-2-4-6-11/h2-9H,10H2,1H3  //  205 : InChI=1S/C16H13ClN2O/c1-19-14-8-7-12(17)9-13(14)16(18-10-15(19)20)11-5-3-2-4-6-11/h2-9H,10H2,1H3
4 : 0  //  205 : 0
34 : InChI=1S/C8H10N2S/c1-2-7-5-6(8(9)11)3-4-10-7/h3-5H,2H2,1H3,(H2,9,11)  //  450 : InChI=1S/C8H10N2S/c1-2-7-5-6(8(9)11)3-4-10-7/h3-5H,2H2,1H3,(H2,9,11)
34 : 0  //  450 : 0
77 : InChI=1S/C23H34O5/c1-4-14(2)23(26)28-20-7-5-6-16-9-8-15(3)19(22(16)20)11-10-18-12-17(24)13-21(25)27-18/h6,8-9,14-15,17-20,22,24H,4-5,7,10-13H2,1-3H3/t14-,15-,17+,18+,19-,20-,22-/m0/s1  //  374 : InChI=1S/C23H34O5/c1-4-14(2)23(26)28-20-7-5-6-16-9-8-15(3)19(22(16)20)11-10-18-12-17(24)13-21(25)27-18/h6,8-9,14-15,17-20,22,24H,4-5,7,10-13H2,1-3H3/t14-,15-,17+,18+,19-,20-,22-/m0

408 : InChI=1S/C12H17N3O/c1-8(2)15-7-12(16)9-3-4-11(14)10(5-9)6-13/h3-5,8,12,15-16H,7,14H2,1-2H3  //  84 : InChI=1S/C12H17N3O/c1-8(2)15-7-12(16)9-3-4-11(14)10(5-9)6-13/h3-5,8,12,15-16H,7,14H2,1-2H3
408 : 0  //  84 : 0
431 : InChI=1S/C18H14Cl4N2O/c19-13-2-1-12(16(21)7-13)10-25-18(9-24-6-5-23-11-24)15-4-3-14(20)8-17(15)22/h1-8,11,18H,9-10H2  //  254 : InChI=1S/C18H14Cl4N2O/c19-13-2-1-12(16(21)7-13)10-25-18(9-24-6-5-23-11-24)15-4-3-14(20)8-17(15)22/h1-8,11,18H,9-10H2
431 : 0  //  254 : 0
436 : InChI=1S/C13H18N4O3/c1-9(18)6-4-5-7-17-12(19)10-11(14-8-15(10)2)16(3)13(17)20/h8H,4-7H2,1-3H3  //  113 : InChI=1S/C13H18N4O3/c1-9(18)6-4-5-7-17-12(19)10-11(14-8-15(10)2)16(3)13(17)20/h8H,4-7H2,1-3H3
436 : 0  //  113 : 0
439 : InChI=1S/C4H6N4O3S2/c1-2(9)6-3-7-8-4(12-3)13(5,10)11/h1H3,(H2,5,10,11)(H,6,7,9)  //  420 : InChI=1S/C4H6N4O3S2/c1-2(9)6-3-7-8-4(12-3)13(5,10)11/h1H3,(H2,5,10,11)(H,6,7,9)
439 : 0  //  420 : 0
454 : InChI=1S/C19H22N2/c1-16-7-9-17(10-8-16)18(19-6-2-3-12-20-19)11-15-21-13-4-5-14-2

893 : InChI=1S/C8H11NO/c9-6-5-7-1-3-8(10)4-2-7/h1-4,10H,5-6,9H2  //  452 : InChI=1S/C8H11NO/c9-6-5-7-1-3-8(10)4-2-7/h1-4,10H,5-6,9H2
893 : 0  //  452 : 0
903 : InChI=1S/C27H36N2O4/c1-4-33-25-17-20(12-13-22(25)27(31)32)18-26(30)28-23(16-19(2)3)21-10-6-7-11-24(21)29-14-8-5-9-15-29/h6-7,10-13,17,19,23H,4-5,8-9,14-16,18H2,1-3H3,(H,28,30)(H,31,32)/t23-/m0/s1  //  401 : InChI=1S/C27H36N2O4/c1-4-33-25-17-20(12-13-22(25)27(31)32)18-26(30)28-23(16-19(2)3)21-10-6-7-11-24(21)29-14-8-5-9-15-29/h6-7,10-13,17,19,23H,4-5,8-9,14-16,18H2,1-3H3,(H,28,30)(H,31,32)/t23-/m0/s1
903 : 0  //  401 : 0
907 : InChI=1S/C9H15NO3S/c1-6(5-14)8(11)10-4-2-3-7(10)9(12)13/h6-7,14H,2-5H2,1H3,(H,12,13)/t6-,7+/m1/s1  //  484 : InChI=1S/C9H15NO3S/c1-6(5-14)8(11)10-4-2-3-7(10)9(12)13/h6-7,14H,2-5H2,1H3,(H,12,13)/t6-,7+/m1/s1
907 : 0  //  484 : 0
915 : InChI=1S/C34H63N5O9/c1-17(2)12-23(37-33(47)31(21(9)10)39-34(48)30(20(7)8)38-27(42)14-19(5)6)25(40)15-28(43)35-22(11)32(46)36-24(13-18(3)4)26(41)16-29(44)45/h17-26,30-31,40-41H,

1276 : InChI=1S/C10H15N5/c1-4-14(5-2)9-6-8(3)13-10-11-7-12-15(9)10/h6-7H,4-5H2,1-3H3  //  25 : InChI=1S/C10H15N5/c1-4-14(5-2)9-6-8(3)13-10-11-7-12-15(9)10/h6-7H,4-5H2,1-3H3
1276 : 0  //  25 : 0
1292 : InChI=1S/C16H23BrN2O3/c1-4-19-9-5-6-11(19)10-18-16(20)14-13(21-2)8-7-12(17)15(14)22-3/h7-8,11H,4-6,9-10H2,1-3H3,(H,18,20)/t11-/m0/s1  //  226 : InChI=1S/C16H23BrN2O3/c1-4-19-9-5-6-11(19)10-18-16(20)14-13(21-2)8-7-12(17)15(14)22-3/h7-8,11H,4-6,9-10H2,1-3H3,(H,18,20)/t11-/m0/s1
1292 : 0  //  226 : 0
1310 : InChI=1S/C14H13N3O5S/c1-10(18)15-11-4-8-14(9-5-11)23(21,22)16-12-2-6-13(7-3-12)17(19)20/h2-9,16H,1H3,(H,15,18)  //  133 : InChI=1S/C14H13N3O5S/c1-10(18)15-11-4-8-14(9-5-11)23(21,22)16-12-2-6-13(7-3-12)17(19)20/h2-9,16H,1H3,(H,15,18)
1310 : 1  //  133 : 1
1339 : InChI=1S/C22H32O3/c1-12-9-15-17-6-5-16(13(2)23)22(17,4)11-19(25)20(15)21(3)8-7-14(24)10-18(12)21/h10,12,15-17,19-20,25H,5-9,11H2,1-4H3/t12-,15-,16+,17-,19-,20+,21-,22+/m0/s1  //  363 : InChI=1S/C22H32O3/c1-12-9-15-17-6-5-16(13(2)23

1700 : InChI=1S/C21H26O2/c1-4-21(22)12-10-19-18-7-5-14-13-15(23-3)6-8-16(14)17(18)9-11-20(19,21)2/h1,6,8,13,17-19,22H,5,7,9-12H2,2-3H3/t17-,18-,19+,20+,21+/m1/s1  //  326 : InChI=1S/C21H26O2/c1-4-21(22)12-10-19-18-7-5-14-13-15(23-3)6-8-16(14)17(18)9-11-20(19,21)2/h1,6,8,13,17-19,22H,5,7,9-12H2,2-3H3/t17-,18-,19+,20+,21+/m1/s1
1700 : 0  //  326 : 0
1703 : InChI=1S/C12H15NO3/c1-8-3-9(2)5-10(4-8)15-7-11-6-13-12(14)16-11/h3-5,11H,6-7H2,1-2H3,(H,13,14)  //  82 : InChI=1S/C12H15NO3/c1-8-3-9(2)5-10(4-8)15-7-11-6-13-12(14)16-11/h3-5,11H,6-7H2,1-2H3,(H,13,14)
1703 : 0  //  82 : 0
1728 : InChI=1S/C19H32N2O5/c1-4-8-14(19(25)26-5-2)20-12(3)17(22)21-15-10-7-6-9-13(15)11-16(21)18(23)24/h12-16,20H,4-11H2,1-3H3,(H,23,24)/t12-,13-,14-,15-,16-/m0/s1  //  298 : InChI=1S/C19H32N2O5/c1-4-8-14(19(25)26-5-2)20-12(3)17(22)21-15-10-7-6-9-13(15)11-16(21)18(23)24/h12-16,20H,4-11H2,1-3H3,(H,23,24)/t12-,13-,14-,15-,16-/m0/s1
1728 : 0  //  298 : 0
1730 : InChI=1S/C7H10ClN3O3/c1-5-9-3-7(11(13)14)10(5)4-6(12)2-8/h3,6

...

3573 : InChI=1S/C15H22N4O3/c1-4-8-18-10-16-13-12(18)14(21)19(15(22)17(13)3)9-6-5-7-11(2)20/h10H,4-9H2,1-3H3  //  196 : InChI=1S/C15H22N4O3/c1-4-8-18-10-16-13-12(18)14(21)19(15(22)17(13)3)9-6-5-7-11(2)20/h10H,4-9H2,1-3H3
3573 : 0  //  196 : 0
3576 : InChI=1S/C14H16N4O3/c1-2-17-8-10(13(20)21)11(19)9-7-15-14(16-12(9)17)18-5-3-4-6-18/h7-8H,2-6H2,1H3,(H,20,21)  //  144 : InChI=1S/C14H16N4O3/c1-2-17-8-10(13(20)21)11(19)9-7-15-14(16-12(9)17)18-5-3-4-6-18/h7-8H,2-6H2,1H3,(H,20,21)
3576 : 0  //  144 : 0
3586 : InChI=1S/C29H33ClN2O2/c1-31(2)27(33)29(24-9-5-3-6-10-24,25-11-7-4-8-12-25)19-22-32-20-17-28(34,18-21-32)23-13-15-26(30)16-14-23/h3-16,34H,17-22H2,1-2H3  //  410 : InChI=1S/C29H33ClN2O2/c1-31(2)27(33)29(24-9-5-3-6-10-24,25-11-7-4-8-12-25)19-22-32-20-17-28(34,18-21-32)23-13-15-26(30)16-14-23/h3-16,34H,17-22H2,1-2H3
3586 : 0  //  410 : 0
3593 : InChI=1S/C15H10O7/c16-7-4-10(19)12-11(5-7)22-15(14(21)13(12)20)6-1-2-8(17)9(18)3-6/h1-5,16-19,21H  //  174 : InChI=1S/C15H10O7/c16-7-4-10(19)12-11(5-

4077 : InChI=1S/C8H8N2O3S/c9-14(11,12)5-7-6-3-1-2-4-8(6)13-10-7/h1-4H,5H2,(H2,9,11,12)  //  467 : InChI=1S/C8H8N2O3S/c9-14(11,12)5-7-6-3-1-2-4-8(6)13-10-7/h1-4H,5H2,(H2,9,11,12)
4077 : 0  //  467 : 0
4080 : InChI=1S/C24H31N5O2/c1-3-27(4-2)16-18-10-7-8-15-28(18)17-22(30)29-21-13-6-5-11-19(21)24(31)26-20-12-9-14-25-23(20)29/h5-6,9,11-14,18H,3-4,7-8,10,15-17H2,1-2H3,(H,26,31)  //  382 : InChI=1S/C24H31N5O2/c1-3-27(4-2)16-18-10-7-8-15-28(18)17-22(30)29-21-13-6-5-11-19(21)24(31)26-20-12-9-14-25-23(20)29/h5-6,9,11-14,18H,3-4,7-8,10,15-17H2,1-2H3,(H,26,31)
4080 : 0  //  382 : 0
4105 : InChI=1S/C13H10N2O4/c16-10-6-5-9(11(17)14-10)15-12(18)7-3-1-2-4-8(7)13(15)19/h1-4,9H,5-6H2,(H,14,16,17)  //  97 : InChI=1S/C13H10N2O4/c16-10-6-5-9(11(17)14-10)15-12(18)7-3-1-2-4-8(7)13(15)19/h1-4,9H,5-6H2,(H,14,16,17)
4105 : 0  //  97 : 0
4110 : InChI=1S/C6H4N2O5/c9-6-2-1-4(7(10)11)3-5(6)8(12)13/h1-3,9H  //  425 : InChI=1S/C6H4N2O5/c9-6-2-1-4(7(10)11)3-5(6)8(12)13/h1-3,9H
4110 : 1  //  425 : 1
4126 : InChI=1S/C9

4555 : InChI=1S/C11H12ClNO3S/c1-13-10(14)6-7-17(15,16)11(13)8-2-4-9(12)5-3-8/h2-5,11H,6-7H2,1H3  //  44 : InChI=1S/C11H12ClNO3S/c1-13-10(14)6-7-17(15,16)11(13)8-2-4-9(12)5-3-8/h2-5,11H,6-7H2,1H3
4555 : 0  //  44 : 0
4558 : InChI=1S/C11H18N2O3/c1-4-6-7(3)11(5-2)8(14)12-10(16)13-9(11)15/h7H,4-6H2,1-3H3,(H2,12,13,14,15,16)  //  62 : InChI=1S/C11H18N2O3/c1-4-6-7(3)11(5-2)8(14)12-10(16)13-9(11)15/h7H,4-6H2,1-3H3,(H2,12,13,14,15,16)
4558 : 0  //  62 : 0
4571 : InChI=1S/C15H23N3OS/c1-5-18(6-2)9-10-19-12-7-8-13-14(11-12)20-15(16-13)17(3)4/h7-8,11H,5-6,9-10H2,1-4H3  //  199 : InChI=1S/C15H23N3OS/c1-5-18(6-2)9-10-19-12-7-8-13-14(11-12)20-15(16-13)17(3)4/h7-8,11H,5-6,9-10H2,1-4H3
4571 : 0  //  199 : 0
4573 : InChI=1S/C7H9N/c8-6-7-4-2-1-3-5-7/h1-5H,6,8H2  //  448 : InChI=1S/C7H9N/c8-6-7-4-2-1-3-5-7/h1-5H,6,8H2
4573 : 0  //  448 : 0
4588 : InChI=1S/C17H12Br2O3/c1-2-13-15(10-5-3-4-6-14(10)22-13)16(20)9-7-11(18)17(21)12(19)8-9/h3-8,21H,2H2,1H3  //  232 : InChI=1S/C17H12Br2O3/c1-2-13-15(10-5-3-4-6-14(

4938 : InChI=1S/C19H22FN3O/c20-17-8-6-16(7-9-17)18(24)4-3-11-22-12-14-23(15-13-22)19-5-1-2-10-21-19/h1-2,5-10H,3-4,11-15H2  //  289 : InChI=1S/C19H22FN3O/c20-17-8-6-16(7-9-17)18(24)4-3-11-22-12-14-23(15-13-22)19-5-1-2-10-21-19/h1-2,5-10H,3-4,11-15H2
4938 : 0  //  289 : 0
4947 : InChI=1S/C19H22FN3O4/c1-10-8-22(6-5-21-10)16-14(20)7-12-15(18(16)27-2)23(11-3-4-11)9-13(17(12)24)19(25)26/h7,9-11,21H,3-6,8H2,1-2H3,(H,25,26)  //  290 : InChI=1S/C19H22FN3O4/c1-10-8-22(6-5-21-10)16-14(20)7-12-15(18(16)27-2)23(11-3-4-11)9-13(17(12)24)19(25)26/h7,9-11,21H,3-6,8H2,1-2H3,(H,25,26)
4947 : 0  //  290 : 0
4950 : InChI=1S/C8H15N7O2S3/c9-6(15-20(12,16)17)1-2-18-3-5-4-19-8(13-5)14-7(10)11/h4H,1-3H2,(H2,9,15)(H2,12,16,17)(H4,10,11,13,14)  //  457 : InChI=1S/C8H15N7O2S3/c9-6(15-20(12,16)17)1-2-18-3-5-4-19-8(13-5)14-7(10)11/h4H,1-3H2,(H2,9,15)(H2,12,16,17)(H4,10,11,13,14)
4950 : 0  //  457 : 0
4953 : InChI=1S/C15H11I4NO4/c16-8-4-7(5-9(17)13(8)21)24-14-10(18)1-6(2-11(14)19)3-12(20)15(22)23/h1-2,4-5,12,21H,3,2

5525 : InChI=1S/C17H21NO3/c1-3-11-6-5-7-12-13-8-9-21-17(4-2,10-14(19)20)16(13)18-15(11)12/h5-7,18H,3-4,8-10H2,1-2H3,(H,19,20)  //  247 : InChI=1S/C17H21NO3/c1-3-11-6-5-7-12-13-8-9-21-17(4-2,10-14(19)20)16(13)18-15(11)12/h5-7,18H,3-4,8-10H2,1-2H3,(H,19,20)
5525 : 0  //  247 : 0
5788 : InChI=1S/C22H24ClN5O2/c23-15-6-7-20-18(14-15)25-22(30)28(20)16-8-12-26(13-9-16)10-3-11-27-19-5-2-1-4-17(19)24-21(27)29/h1-2,4-7,14,16H,3,8-13H2,(H,24,29)(H,25,30)  //  350 : InChI=1S/C22H24ClN5O2/c23-15-6-7-20-18(14-15)25-22(30)28(20)16-8-12-26(13-9-16)10-3-11-27-19-5-2-1-4-17(19)24-21(27)29/h1-2,4-7,14,16H,3,8-13H2,(H,24,29)(H,25,30)
5788 : 0  //  350 : 0
5803 : InChI=1S/C32H40BrN5O5/c1-16(2)12-24-29(40)37-11-7-10-25(37)32(42)38(24)30(41)31(43-32,17(3)4)35-28(39)18-13-20-19-8-6-9-22-26(19)21(27(33)34-22)14-23(20)36(5)15-18/h6,8-9,13,16-18,23-25,34,42H,7,10-12,14-15H2,1-5H3,(H,35,39)/t18-,23-,24+,25+,31-,32+/m1/s1  //  415 : InChI=1S/C32H40BrN5O5/c1-16(2)12-24-29(40)37-11-7-10-25(37)32(42)38(24)30(41)31(43

In [164]:
len(num),len(counter1),len(counter2)

(424, 11, 11)

In [165]:
d3_ = d3.drop(num)
d3_.reset_index(inplace=True)
d3_.drop("index",axis = 1,inplace = True)
d3_["Toxicity"].value_counts()

Toxicity
0    53
1    19
Name: count, dtype: int64

In [166]:
#Have overlap index, so weird....
#Some conpound's mol will be None when we 
counter1 = set(counter1)
d_total_ = d_total.drop(counter1)
d_total_.reset_index(inplace=True)
d_total_.drop("index",axis = 1,inplace = True)
d_total_["Mol"] = [Chem.MolFromInchi(x) for x in d_total_["Inchi"]]
d_total_["Toxicity"].value_counts()

[12:04:30] WARNING: not removing hydrogen atom without neighbors
[12:04:30] Explicit valence for atom # 0 Cl, 5, is greater than permitted
[12:04:30] ERROR: Explicit valence for atom # 0 Cl, 5, is greater than permitted

[12:04:31] WARNING:  Problems/mismatches: Mobile-H( Hydrogens: Locations or number, Number; Charge(s): Do not match; Proton balance: Does not match)

[12:04:31] Explicit valence for atom # 0 Cl, 3, is greater than permitted
[12:04:31] ERROR: Explicit valence for atom # 0 Cl, 3, is greater than permitted

[12:04:32] WARNING: not removing hydrogen atom without neighbors
[12:04:32] WARNING: not removing hydrogen atom without neighbors
[12:04:32] WARNING: not removing hydrogen atom without neighbors
[12:04:32] Explicit valence for atom # 0 Cl, 5, is greater than permitted
[12:04:32] ERROR: Explicit valence for atom # 0 Cl, 5, is greater than permitted

[12:04:32] WARNING:  Problems/mismatches: Mobile-H( Hydrogens: Locations or number; Mobile-H groups: Missing, Attachment p

Toxicity
0    5788
1    1495
Name: count, dtype: int64

In [167]:
d_total_ = pd.concat([d_total_,d3_],axis = 0)
d_total_.reset_index(inplace=True)
d_total_.drop("index",axis = 1,inplace = True)
d_total_["Toxicity"].value_counts()

Toxicity
0    5841
1    1514
Name: count, dtype: int64

In [168]:
#combine d_total and cp and ge into dtotal(same assay)
#delete the overlap part
num = []
counter1 = []
counter2 = []
for i in range(len(d_total_)):
    for y in range(len(d5)):
        if(d_total_["Inchi"][i] == d5["Inchi"][y]):
            num.append(y)
            print(i,":",d_total_["Inchi"][i]," // ",y,":",d5["Inchi"][y])
            print(i,":",d_total_["Toxicity"][i]," // ",y,":",d5["Toxicity"][y])
            if d_total_["Toxicity"][i] != d5["Toxicity"][y]:
                counter1.append(i)
                counter2.append(y)

4 : InChI=1S/C16H13ClN2O/c1-19-14-8-7-12(17)9-13(14)16(18-10-15(19)20)11-5-3-2-4-6-11/h2-9H,10H2,1H3  //  282 : InChI=1S/C16H13ClN2O/c1-19-14-8-7-12(17)9-13(14)16(18-10-15(19)20)11-5-3-2-4-6-11/h2-9H,10H2,1H3
4 : 0  //  282 : 0
15 : InChI=1S/C13H6Cl6O2/c14-6-2-8(16)12(20)4(10(6)18)1-5-11(19)7(15)3-9(17)13(5)21/h2-3,20-21H,1H2  //  175 : InChI=1S/C13H6Cl6O2/c14-6-2-8(16)12(20)4(10(6)18)1-5-11(19)7(15)3-9(17)13(5)21/h2-3,20-21H,1H2
15 : 1  //  175 : 1
15 : InChI=1S/C13H6Cl6O2/c14-6-2-8(16)12(20)4(10(6)18)1-5-11(19)7(15)3-9(17)13(5)21/h2-3,20-21H,1H2  //  176 : InChI=1S/C13H6Cl6O2/c14-6-2-8(16)12(20)4(10(6)18)1-5-11(19)7(15)3-9(17)13(5)21/h2-3,20-21H,1H2
15 : 1  //  176 : 1
32 : InChI=1S/C19H26O2/c1-18-9-7-13(20)11-12(18)3-4-14-15-5-6-17(21)19(15,2)10-8-16(14)18/h11,14-16H,3-10H2,1-2H3/t14-,15-,16-,18-,19-/m0/s1  //  420 : InChI=1S/C19H26O2/c1-18-9-7-13(20)11-12(18)3-4-14-15-5-6-17(21)19(15,2)10-8-16(14)18/h11,14-16H,3-10H2,1-2H3/t14-,15-,16-,18-,19-/m0/s1
32 : 0  //  420 : 0
34 : InChI=1

302 : InChI=1S/C46H65N13O12S2/c47-17-5-4-9-29(40(65)52-22-38(51)63)54-45(70)35-10-6-18-59(35)46(71)34-24-73-72-23-28(48)39(64)55-31(20-26-11-13-27(60)14-12-26)43(68)56-32(19-25-7-2-1-3-8-25)42(67)53-30(15-16-36(49)61)41(66)57-33(21-37(50)62)44(69)58-34/h1-3,7-8,11-14,28-35,60H,4-6,9-10,15-24,47-48H2,(H2,49,61)(H2,50,62)(H2,51,63)(H,52,65)(H,53,67)(H,54,70)(H,55,64)(H,56,68)(H,57,66)(H,58,69)/t28-,29-,30-,31-,32-,33-,34-,35-/m0/s1  //  625 : InChI=1S/C46H65N13O12S2/c47-17-5-4-9-29(40(65)52-22-38(51)63)54-45(70)35-10-6-18-59(35)46(71)34-24-73-72-23-28(48)39(64)55-31(20-26-11-13-27(60)14-12-26)43(68)56-32(19-25-7-2-1-3-8-25)42(67)53-30(15-16-36(49)61)41(66)57-33(21-37(50)62)44(69)58-34/h1-3,7-8,11-14,28-35,60H,4-6,9-10,15-24,47-48H2,(H2,49,61)(H2,50,62)(H2,51,63)(H,52,65)(H,53,67)(H,54,70)(H,55,64)(H,56,68)(H,57,66)(H,58,69)/t28-,29-,30-,31-,32-,33-,34-,35-/m0/s1
302 : 0  //  625 : 0
308 : InChI=1S/C12H16O7/c13-5-8-9(15)10(16)11(17)12(19-8)18-7-3-1-6(14)2-4-7/h1-4,8-17H,5H2/t8-,9-,10+,11-

579 : InChI=1S/C6H14O6S2/c1-13(7,8)11-5-3-4-6-12-14(2,9)10/h3-6H2,1-2H3  //  652 : InChI=1S/C6H14O6S2/c1-13(7,8)11-5-3-4-6-12-14(2,9)10/h3-6H2,1-2H3
579 : 0  //  652 : 0
581 : InChI=1S/C10H13NO2/c1-3-13-10-6-4-9(5-7-10)11-8(2)12/h4-7H,3H2,1-2H3,(H,11,12)  //  26 : InChI=1S/C10H13NO2/c1-3-13-10-6-4-9(5-7-10)11-8(2)12/h4-7H,3H2,1-2H3,(H,11,12)
581 : 0  //  26 : 0
582 : InChI=1S/C12H19N3O/c1-9(2)15-12(16)11-6-4-10(5-7-11)8-14-13-3/h4-7,9,13-14H,8H2,1-3H3,(H,15,16)  //  122 : InChI=1S/C12H19N3O/c1-9(2)15-12(16)11-6-4-10(5-7-11)8-14-13-3/h4-7,9,13-14H,8H2,1-3H3,(H,15,16)
582 : 0  //  122 : 0
588 : InChI=1S/C10H16N2O/c1-2-7(1)9(8-3-4-8)12-10-11-5-6-13-10/h7-9H,1-6H2,(H,11,12)  //  40 : InChI=1S/C10H16N2O/c1-2-7(1)9(8-3-4-8)12-10-11-5-6-13-10/h7-9H,1-6H2,(H,11,12)
588 : 0  //  40 : 0
591 : InChI=1S/C15H26O/c1-13(2)7-5-8-14(3)9-6-10-15(4)11-12-16/h7,9,11,16H,5-6,8,10,12H2,1-4H3/b14-9+,15-11+  //  279 : InChI=1S/C15H26O/c1-13(2)7-5-8-14(3)9-6-10-15(4)11-12-16/h7,9,11,16H,5-6,8,10,12H2,1-4H3/b14

837 : InChI=1S/C27H32N4O7S/c1-18(2)20-14-19(36-3)15-22-25(20)27(33)31(39(22,34)35)17-38-23-16-24(32)30-11-7-8-21(26(30)28-23)37-13-12-29-9-5-4-6-10-29/h7-8,11,14-16,18H,4-6,9-10,12-13,17H2,1-3H3  //  577 : InChI=1S/C27H32N4O7S/c1-18(2)20-14-19(36-3)15-22-25(20)27(33)31(39(22,34)35)17-38-23-16-24(32)30-11-7-8-21(26(30)28-23)37-13-12-29-9-5-4-6-10-29/h7-8,11,14-16,18H,4-6,9-10,12-13,17H2,1-3H3
837 : 0  //  577 : 0
847 : InChI=1S/C18H30O2/c1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-17-18(19)20/h3-4,6-7,9-10H,2,5,8,11-17H2,1H3,(H,19,20)/b4-3-,7-6-,10-9-  //  387 : InChI=1S/C18H30O2/c1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-17-18(19)20/h3-4,6-7,9-10H,2,5,8,11-17H2,1H3,(H,19,20)/b4-3-,7-6-,10-9-
847 : 0  //  387 : 0
848 : InChI=1S/C21H45N3/c1-6-10-12-19(8-3)14-23-16-21(5,22)17-24(18-23)15-20(9-4)13-11-7-2/h19-20H,6-18,22H2,1-5H3  //  491 : InChI=1S/C21H45N3/c1-6-10-12-19(8-3)14-23-16-21(5,22)17-24(18-23)15-20(9-4)13-11-7-2/h19-20H,6-18,22H2,1-5H3
848 : 0  //  491 : 0
851 : InChI=1S/C12H15NO3/c1-12

1024 : InChI=1S/C12H11N7/c13-9-7(6-4-2-1-3-5-6)16-8-10(14)18-12(15)19-11(8)17-9/h1-5H,(H6,13,14,15,17,18,19)  //  91 : InChI=1S/C12H11N7/c13-9-7(6-4-2-1-3-5-6)16-8-10(14)18-12(15)19-11(8)17-9/h1-5H,(H6,13,14,15,17,18,19)
1024 : 0  //  91 : 0
1026 : InChI=1S/C6H12N3PS/c11-10(7-1-2-7,8-3-4-8)9-5-6-9/h1-6H2  //  649 : InChI=1S/C6H12N3PS/c11-10(7-1-2-7,8-3-4-8)9-5-6-9/h1-6H2
1026 : 0  //  649 : 0
1051 : InChI=1S/C22H32O3/c1-13-11-16-17(20(3)8-5-15(24)12-19(13)20)6-9-21(4)18(16)7-10-22(21,25)14(2)23/h12-13,16-18,25H,5-11H2,1-4H3/t13-,16+,17-,18-,20+,21-,22-/m0/s1  //  521 : InChI=1S/C22H32O3/c1-13-11-16-17(20(3)8-5-15(24)12-19(13)20)6-9-21(4)18(16)7-10-22(21,25)14(2)23/h12-13,16-18,25H,5-11H2,1-4H3/t13-,16+,17-,18-,20+,21-,22-/m0/s1
1051 : 0  //  521 : 0
1057 : InChI=1S/C19H24N2O2/c22-18-13-20(19(23)15-7-2-1-3-8-15)12-17-16-9-5-4-6-14(16)10-11-21(17)18/h4-6,9,15,17H,1-3,7-8,10-13H2  //  417 : InChI=1S/C19H24N2O2/c22-18-13-20(19(23)15-7-2-1-3-8-15)12-17-16-9-5-4-6-14(16)10-11-21(17)18/h4-6,9

...

4997 : InChI=1S/C13H16N2O/c1-2-4-12(11(3-1)10-5-6-10)16-9-13-14-7-8-15-13/h1-4,10H,5-9H2,(H,14,15)  //  157 : InChI=1S/C13H16N2O/c1-2-4-12(11(3-1)10-5-6-10)16-9-13-14-7-8-15-13/h1-4,10H,5-9H2,(H,14,15)
4997 : 0  //  157 : 0
5008 : InChI=1S/C10H16N2O3S/c13-8(14)4-2-1-3-7-9-6(5-16-7)11-10(15)12-9/h6-7,9H,1-5H2,(H,13,14)(H2,11,12,15)/t6-,7-,9-/m0/s1  //  41 : InChI=1S/C10H16N2O3S/c13-8(14)4-2-1-3-7-9-6(5-16-7)11-10(15)12-9/h6-7,9H,1-5H2,(H,13,14)(H2,11,12,15)/t6-,7-,9-/m0/s1
5008 : 0  //  41 : 0
5024 : InChI=1S/C12H9N3O5/c16-9-3-1-8(2-4-9)12(17)14-13-7-10-5-6-11(20-10)15(18)19/h1-7,16H,(H,14,17)  //  137 : InChI=1S/C12H9N3O5/c16-9-3-1-8(2-4-9)12(17)14-13-7-10-5-6-11(20-10)15(18)19/h1-7,16H,(H,14,17)
5024 : 1  //  137 : 1
5030 : InChI=1S/C10H16N8S2/c1-14-9(16-6-11)15-2-3-19-4-7-5-20-10(17-7)18-8(12)13/h5H,2-4H2,1H3,(H2,14,15,16)(H4,12,13,17,18)  //  43 : InChI=1S/C10H16N8S2/c1-14-9(16-6-11)15-2-3-19-4-7-5-20-10(17-7)18-8(12)13/h5H,2-4H2,1H3,(H2,14,15,16)(H4,12,13,17,18)
5030 : 0  //  43 : 

5305 : InChI=1S/C7H8N4O2/c1-10-5-4(8-3-9-5)6(12)11(2)7(10)13/h3H,1-2H3,(H,8,9)  //  693 : InChI=1S/C7H8N4O2/c1-10-5-4(8-3-9-5)6(12)11(2)7(10)13/h3H,1-2H3,(H,8,9)
5305 : 0  //  693 : 0
5330 : InChI=1S/C21H27N5O4S/c1-15-13-24-19(14-23-15)20(27)22-12-11-16-7-9-18(10-8-16)31(29,30)26-21(28)25-17-5-3-2-4-6-17/h7-10,13-14,17H,2-6,11-12H2,1H3,(H,22,27)(H2,25,26,28)  //  474 : InChI=1S/C21H27N5O4S/c1-15-13-24-19(14-23-15)20(27)22-12-11-16-7-9-18(10-8-16)31(29,30)26-21(28)25-17-5-3-2-4-6-17/h7-10,13-14,17H,2-6,11-12H2,1H3,(H,22,27)(H2,25,26,28)
5330 : 0  //  474 : 0
5333 : InChI=1S/C14H18N2O/c1-9(2)13-12(14(17)10(3)4)11-7-5-6-8-16(11)15-13/h5-10H,1-4H3  //  207 : InChI=1S/C14H18N2O/c1-9(2)13-12(14(17)10(3)4)11-7-5-6-8-16(11)15-13/h5-10H,1-4H3
5333 : 0  //  207 : 0
5350 : InChI=1S/C18H21NO/c20-18(15-7-3-1-4-8-15,16-9-5-2-6-10-16)17-11-13-19-14-12-17/h1-10,17,19-20H,11-14H2  //  369 : InChI=1S/C18H21NO/c20-18(15-7-3-1-4-8-15,16-9-5-2-6-10-16)17-11-13-19-14-12-17/h1-10,17,19-20H,11-14H2
5350 : 0  

5950 : InChI=1S/C12H9N3O/c1-8-11(9-2-4-14-5-3-9)6-10(7-13)12(16)15-8/h2-6H,1H3,(H,15,16)  //  136 : InChI=1S/C12H9N3O/c1-8-11(9-2-4-14-5-3-9)6-10(7-13)12(16)15-8/h2-6H,1H3,(H,15,16)
5950 : 0  //  136 : 0
5963 : InChI=1S/C10H15N5O3/c11-10-13-8-7(9(18)14-10)12-5-15(8)2-1-6(3-16)4-17/h5-6,16-17H,1-4H2,(H3,11,13,14,18)  //  39 : InChI=1S/C10H15N5O3/c11-10-13-8-7(9(18)14-10)12-5-15(8)2-1-6(3-16)4-17/h5-6,16-17H,1-4H2,(H3,11,13,14,18)
5963 : 0  //  39 : 0
5982 : InChI=1S/C26H37N5O2/c1-5-11-30-17-19(25(32)31(26(33)27-6-2)13-8-12-29(3)4)14-21-20-9-7-10-22-24(20)18(16-28-22)15-23(21)30/h5,7,9-10,16,19,21,23,28H,1,6,8,11-15,17H2,2-4H3,(H,27,33)/t19-,21-,23-/m1/s1  //  570 : InChI=1S/C26H37N5O2/c1-5-11-30-17-19(25(32)31(26(33)27-6-2)13-8-12-29(3)4)14-21-20-9-7-10-22-24(20)18(16-28-22)15-23(21)30/h5,7,9-10,16,19,21,23,28H,1,6,8,11-15,17H2,2-4H3,(H,27,33)/t19-,21-,23-/m1/s1
5982 : 0  //  570 : 0
6113 : InChI=1S/C14H20N2O2/c1-10(2)16-8-11(17)9-18-14-5-3-4-13-12(14)6-7-15-13/h3-7,10-11,15-17H,8-9H2,1

6994 : InChI=1S/C16H8Cl4N2O2/c17-9-5-13(19)15(21-7-9)23-11-1-2-12(4-3-11)24-16-14(20)6-10(18)8-22-16/h1-8H  //  321 : InChI=1S/C16H8Cl4N2O2/c17-9-5-13(19)15(21-7-9)23-11-1-2-12(4-3-11)24-16-14(20)6-10(18)8-22-16/h1-8H
6994 : 1  //  321 : 1
7050 : InChI=1S/C13H17N3O2/c1-3-4-5-9-6-7-10-11(8-9)15-12(14-10)16-13(17)18-2/h6-8H,3-5H2,1-2H3,(H2,14,15,16,17)  //  162 : InChI=1S/C13H17N3O2/c1-3-4-5-9-6-7-10-11(8-9)15-12(14-10)16-13(17)18-2/h6-8H,3-5H2,1-2H3,(H2,14,15,16,17)
7050 : 1  //  162 : 1
7054 : InChI=1S/C24H20N6O3/c1-2-33-24-25-20-9-5-8-19(23(31)32)21(20)30(24)14-15-10-12-16(13-11-15)17-6-3-4-7-18(17)22-26-28-29-27-22/h3-13H,2,14H2,1H3,(H,31,32)(H,26,27,28,29)  //  539 : InChI=1S/C24H20N6O3/c1-2-33-24-25-20-9-5-8-19(23(31)32)21(20)30(24)14-15-10-12-16(13-11-15)17-6-3-4-7-18(17)22-26-28-29-27-22/h3-13H,2,14H2,1H3,(H,31,32)(H,26,27,28,29)
7054 : 1  //  539 : 1
7065 : InChI=1S/C16H13N3O3/c1-22-16(21)19-15-17-12-8-7-11(9-13(12)18-15)14(20)10-5-3-2-4-6-10/h2-9H,1H3,(H2,17,18,19,21)  //  283 

7316 : InChI=1S/C17H11N5/c18-9-13-1-5-15(6-2-13)17(22-12-20-11-21-22)16-7-3-14(10-19)4-8-16/h1-8,11-12,17H  //  322 : InChI=1S/C17H11N5/c18-9-13-1-5-15(6-2-13)17(22-12-20-11-21-22)16-7-3-14(10-19)4-8-16/h1-8,11-12,17H
7316 : 0  //  322 : 0
7317 : InChI=1S/C17H14O4S/c1-22(19,20)14-9-7-12(8-10-14)15-11-21-17(18)16(15)13-5-3-2-4-6-13/h2-10H,11H2,1H3  //  329 : InChI=1S/C17H14O4S/c1-22(19,20)14-9-7-12(8-10-14)15-11-21-17(18)16(15)13-5-3-2-4-6-13/h2-10H,11H2,1H3
7317 : 0  //  329 : 0
7318 : InChI=1S/C18H20FN3O4/c1-10-9-26-17-14-11(16(23)12(18(24)25)8-22(10)14)7-13(19)15(17)21-5-3-20(2)4-6-21/h7-8,10H,3-6,9H2,1-2H3,(H,24,25)  //  364 : InChI=1S/C18H20FN3O4/c1-10-9-26-17-14-11(16(23)12(18(24)25)8-22(10)14)7-13(19)15(17)21-5-3-20(2)4-6-21/h7-8,10H,3-6,9H2,1-2H3,(H,24,25)
7318 : 0  //  364 : 0
7319 : InChI=1S/C18H22O2/c1-18-9-8-14-13-5-3-12(19)10-11(13)2-4-15(14)16(18)6-7-17(18)20/h3,5,10,14-16,19H,2,4,6-9H2,1H3/t14-,15-,16+,18+/m1/s1  //  372 : InChI=1S/C18H22O2/c1-18-9-8-14-13-5-3-12(19)10-11

In [169]:
len(num),len(counter1),len(counter2)

(692, 17, 17)

In [170]:
d5_ = d5.drop(num)
d5_.reset_index(inplace=True)
d5_.drop("index",axis = 1,inplace = True)
d5_["Toxicity"].value_counts()

Toxicity
0    58
1    19
Name: count, dtype: int64

In [171]:
#Have overlap index, so weird....
#Some conpound's mol will be None when we 
counter1 = set(counter1)
d_total__ = d_total_.drop(counter1)
d_total__.reset_index(inplace=True)
d_total__.drop("index",axis = 1,inplace = True)
d_total__["Mol"] = [Chem.MolFromInchi(x) for x in d_total__["Inchi"]]
d_total__["Toxicity"].value_counts()

[12:05:14] WARNING: not removing hydrogen atom without neighbors
[12:05:14] Explicit valence for atom # 0 Cl, 5, is greater than permitted
[12:05:14] ERROR: Explicit valence for atom # 0 Cl, 5, is greater than permitted

[12:05:14] WARNING:  Problems/mismatches: Mobile-H( Hydrogens: Locations or number, Number; Charge(s): Do not match; Proton balance: Does not match)

[12:05:15] Explicit valence for atom # 0 Cl, 3, is greater than permitted
[12:05:15] ERROR: Explicit valence for atom # 0 Cl, 3, is greater than permitted

[12:05:15] WARNING: not removing hydrogen atom without neighbors
[12:05:15] WARNING: not removing hydrogen atom without neighbors
[12:05:15] WARNING: not removing hydrogen atom without neighbors
[12:05:15] Explicit valence for atom # 0 Cl, 5, is greater than permitted
[12:05:15] ERROR: Explicit valence for atom # 0 Cl, 5, is greater than permitted

[12:05:15] WARNING:  Problems/mismatches: Mobile-H( Hydrogens: Locations or number; Mobile-H groups: Missing, Attachment p

Toxicity
0    5835
1    1503
Name: count, dtype: int64

In [172]:
d_total__ = pd.concat([d_total__,d5_],axis = 0)
d_total__.reset_index(inplace=True)
d_total__.drop("index",axis = 1,inplace = True)
d_total__["Toxicity"].value_counts()

Toxicity
0    5893
1    1522
Name: count, dtype: int64

In [173]:
#combine d2 and total
#zhang
#delete the overlap part
num = []
counter1 = []
counter2 = []
for i in range(len(d_total__)):
    for y in range(len(d2)):
        if(d_total__["Inchi"][i] == d2["Inchi"][y]):
            num.append(y)
            print(i,":",d_total__["Inchi"][i]," // ",y,":",d2["Inchi"][y])
            print(i,":",d_total__["Toxicity"][i]," // ",y,":",d2["Toxicity"][y])
            if d_total__["Toxicity"][i] != d2["Toxicity"][y]:
                counter1.append(i)
                counter2.append(y)

128 : InChI=1S/C18H29N3O5/c1-18(2,3)19-11-15(22)12-8-13(25-16(23)20(4)5)10-14(9-12)26-17(24)21(6)7/h8-10,15,19,22H,11H2,1-7H3  //  0 : InChI=1S/C18H29N3O5/c1-18(2,3)19-11-15(22)12-8-13(25-16(23)20(4)5)10-14(9-12)26-17(24)21(6)7/h8-10,15,19,22H,11H2,1-7H3
128 : 0  //  0 : 0
169 : InChI=1S/C9H7Cl2N5/c10-4-1-2-6(11)5(3-4)7-14-8(12)16-9(13)15-7/h1-3H,(H4,12,13,14,15,16)  //  2 : InChI=1S/C9H7Cl2N5/c10-4-1-2-6(11)5(3-4)7-14-8(12)16-9(13)15-7/h1-3H,(H4,12,13,14,15,16)
169 : 0  //  2 : 0
238 : InChI=1S/C34H54O8/c1-9-25(31-21(6)18-34(11-3,42-31)26-16-17-33(40,10-2)23(8)41-26)30(37)22(7)28(35)19(4)12-14-24-15-13-20(5)29(36)27(24)32(38)39/h13,15,19,21-23,25-26,28,31,35-36,40H,9-12,14,16-18H2,1-8H3,(H,38,39)/t19-,21+,22+,23+,25+,26-,28+,31+,33-,34+/m1/s1  //  3 : InChI=1S/C34H54O8/c1-9-25(31-21(6)18-34(11-3,42-31)26-16-17-33(40,10-2)23(8)41-26)30(37)22(7)28(35)19(4)12-14-24-15-13-20(5)29(36)27(24)32(38)39/h13,15,19,21-23,25-26,28,31,35-36,40H,9-12,14,16-18H2,1-8H3,(H,38,39)/t19-,21+,22+,23+,25+,2

1646 : InChI=1S/C14H9ClN2O3S/c15-7-3-4-9-8(6-7)11(13(19)17(9)14(16)20)12(18)10-2-1-5-21-10/h1-6,11H,(H2,16,20)  //  34 : InChI=1S/C14H9ClN2O3S/c15-7-3-4-9-8(6-7)11(13(19)17(9)14(16)20)12(18)10-2-1-5-21-10/h1-6,11H,(H2,16,20)
1646 : 1  //  34 : 1
1701 : InChI=1S/C15H21N3O/c1-11(5-3-7-16)18-14-10-13(19-2)9-12-6-4-8-17-15(12)14/h4,6,8-11,18H,3,5,7,16H2,1-2H3  //  35 : InChI=1S/C15H21N3O/c1-11(5-3-7-16)18-14-10-13(19-2)9-12-6-4-8-17-15(12)14/h4,6,8-11,18H,3,5,7,16H2,1-2H3
1701 : 0  //  35 : 0
1705 : InChI=1S/C9H9Cl2N3O/c10-6-2-1-3-7(11)5(6)4-8(15)14-9(12)13/h1-3H,4H2,(H4,12,13,14,15)  //  36 : InChI=1S/C9H9Cl2N3O/c10-6-2-1-3-7(11)5(6)4-8(15)14-9(12)13/h1-3H,4H2,(H4,12,13,14,15)
1705 : 0  //  36 : 0
1709 : InChI=1S/C15H22N2O/c1-11-7-6-8-12(2)14(11)16-15(18)13-9-4-5-10-17(13)3/h6-8,13H,4-5,9-10H2,1-3H3,(H,16,18)  //  37 : InChI=1S/C15H22N2O/c1-11-7-6-8-12(2)14(11)16-15(18)13-9-4-5-10-17(13)3/h6-8,13H,4-5,9-10H2,1-3H3,(H,16,18)
1709 : 0  //  37 : 0
1715 : InChI=1S/C17H21NO3/c1-12(17(21)14-4-8

2493 : InChI=1S/C22H24ClN3O/c1-25-13-4-5-18(12-14-25)26-22(27)20-7-3-2-6-19(20)21(24-26)15-16-8-10-17(23)11-9-16/h2-3,6-11,18H,4-5,12-15H2,1H3  //  66 : InChI=1S/C22H24ClN3O/c1-25-13-4-5-18(12-14-25)26-22(27)20-7-3-2-6-19(20)21(24-26)15-16-8-10-17(23)11-9-16/h2-3,6-11,18H,4-5,12-15H2,1H3
2493 : 0  //  66 : 0
2515 : InChI=1S/C13H17N/c1-4-10-14(3)12(2)11-13-8-6-5-7-9-13/h1,5-9,12H,10-11H2,2-3H3/t12-/m1/s1  //  67 : InChI=1S/C13H17N/c1-4-10-14(3)12(2)11-13-8-6-5-7-9-13/h1,5-9,12H,10-11H2,2-3H3/t12-/m1/s1
2515 : 0  //  67 : 0
2588 : InChI=1S/C15H13N3O4S/c1-18-13(15(20)17-12-8-4-5-9-16-12)14(19)10-6-2-3-7-11(10)23(18,21)22/h2-9,13H,1H3,(H,16,17,20)  //  69 : InChI=1S/C15H13N3O4S/c1-18-13(15(20)17-12-8-4-5-9-16-12)14(19)10-6-2-3-7-11(10)23(18,21)22/h2-9,13H,1H3,(H,16,17,20)
2588 : 1  //  69 : 1
2614 : InChI=1S/C17H19N3O/c1-13-5-7-14(8-6-13)20(12-17-18-9-10-19-17)15-3-2-4-16(21)11-15/h2-8,11,21H,9-10,12H2,1H3,(H,18,19)  //  70 : InChI=1S/C17H19N3O/c1-13-5-7-14(8-6-13)20(12-17-18-9-10-19-17)15

3713 : InChI=1S/C17H18F3NO/c1-21-12-11-16(13-5-3-2-4-6-13)22-15-9-7-14(8-10-15)17(18,19)20/h2-10,16,21H,11-12H2,1H3  //  101 : InChI=1S/C17H18F3NO/c1-21-12-11-16(13-5-3-2-4-6-13)22-15-9-7-14(8-10-15)17(18,19)20/h2-10,16,21H,11-12H2,1H3
3713 : 1  //  101 : 1
3736 : InChI=1S/C23H25N5O5/c1-30-18-11-14-15(12-19(18)31-2)25-23(26-21(14)24)28-9-7-27(8-10-28)22(29)20-13-32-16-5-3-4-6-17(16)33-20/h3-6,11-12,20H,7-10,13H2,1-2H3,(H2,24,25,26)  //  102 : InChI=1S/C23H25N5O5/c1-30-18-11-14-15(12-19(18)31-2)25-23(26-21(14)24)28-9-7-27(8-10-28)22(29)20-13-32-16-5-3-4-6-17(16)33-20/h3-6,11-12,20H,7-10,13H2,1-2H3,(H2,24,25,26)
3736 : 0  //  102 : 0
3752 : InChI=1S/C32H39NO4/c1-31(2,30(35)36)25-17-15-24(16-18-25)29(34)14-9-21-33-22-19-28(20-23-33)32(37,26-10-5-3-6-11-26)27-12-7-4-8-13-27/h3-8,10-13,15-18,28-29,34,37H,9,14,19-23H2,1-2H3,(H,35,36)  //  103 : InChI=1S/C32H39NO4/c1-31(2,30(35)36)25-17-15-24(16-18-25)29(34)14-9-21-33-22-19-28(20-23-33)32(37,26-10-5-3-6-11-26)27-12-7-4-8-13-27/h3-8,10-13,15-1

4856 : InChI=1S/C26H28Cl2N4O4/c1-19(33)31-10-12-32(13-11-31)21-3-5-22(6-4-21)34-15-23-16-35-26(36-23,17-30-9-8-29-18-30)24-7-2-20(27)14-25(24)28/h2-9,14,18,23H,10-13,15-17H2,1H3/t23-,26-/m0/s1  //  133 : InChI=1S/C26H28Cl2N4O4/c1-19(33)31-10-12-32(13-11-31)21-3-5-22(6-4-21)34-15-23-16-35-26(36-23,17-30-9-8-29-18-30)24-7-2-20(27)14-25(24)28/h2-9,14,18,23H,10-13,15-17H2,1H3/t23-,26-/m0/s1
4856 : 1  //  133 : 1
4881 : InChI=1S/C24H28N2O5/c1-2-31-24(30)20(14-12-17-8-4-3-5-9-17)25-19-15-13-18-10-6-7-11-21(18)26(23(19)29)16-22(27)28/h3-11,19-20,25H,2,12-16H2,1H3,(H,27,28)/t19-,20-/m0/s1  //  134 : InChI=1S/C24H28N2O5/c1-2-31-24(30)20(14-12-17-8-4-3-5-9-17)25-19-15-13-18-10-6-7-11-21(18)26(23(19)29)16-22(27)28/h3-11,19-20,25H,2,12-16H2,1H3,(H,27,28)/t19-,20-/m0/s1
4881 : 0  //  134 : 0
4914 : InChI=1S/C6H8ClN7O/c7-2-4(9)13-3(8)1(12-2)5(15)14-6(10)11/h(H4,8,9,13)(H4,10,11,14,15)  //  135 : InChI=1S/C6H8ClN7O/c7-2-4(9)13-3(8)1(12-2)5(15)14-6(10)11/h(H4,8,9,13)(H4,10,11,14,15)
4914 : 0  //  135 

In [174]:
len(num),len(counter1),len(counter2)

(141, 3, 3)

In [175]:
d2_ = d2.drop(num)
d2_.reset_index(inplace=True)
d2_.drop("index",axis = 1,inplace = True)
d2_["Toxicity"].value_counts()

Toxicity
0    81
1    22
Name: count, dtype: int64

In [176]:
#Have overlap index, so weird....
#Some conpound's mol will be None when we 
counter1 = set(counter1)
d_total___ = d_total__.drop(counter1)
d_total___.reset_index(inplace=True)
d_total___.drop("index",axis = 1,inplace = True)
d_total___["Mol"] = [Chem.MolFromInchi(x) for x in d_total___["Inchi"]]
d_total___["Toxicity"].value_counts()

[12:05:29] WARNING: not removing hydrogen atom without neighbors
[12:05:29] Explicit valence for atom # 0 Cl, 5, is greater than permitted
[12:05:29] ERROR: Explicit valence for atom # 0 Cl, 5, is greater than permitted

[12:05:30] WARNING:  Problems/mismatches: Mobile-H( Hydrogens: Locations or number, Number; Charge(s): Do not match; Proton balance: Does not match)

[12:05:30] Explicit valence for atom # 0 Cl, 3, is greater than permitted
[12:05:30] ERROR: Explicit valence for atom # 0 Cl, 3, is greater than permitted

[12:05:31] WARNING: not removing hydrogen atom without neighbors
[12:05:31] WARNING: not removing hydrogen atom without neighbors
[12:05:31] WARNING: not removing hydrogen atom without neighbors
[12:05:31] Explicit valence for atom # 0 Cl, 5, is greater than permitted
[12:05:31] ERROR: Explicit valence for atom # 0 Cl, 5, is greater than permitted

[12:05:31] WARNING:  Problems/mismatches: Mobile-H( Hydrogens: Locations or number; Mobile-H groups: Missing, Attachment p

Toxicity
0    5890
1    1522
Name: count, dtype: int64

In [177]:
d_final = pd.concat([d_total___,d2_],axis = 0)
d_final.reset_index(inplace=True)
d_final.drop("index",axis = 1,inplace = True)
d_final["Toxicity"].value_counts()

Toxicity
0    5971
1    1544
Name: count, dtype: int64

In [178]:
# d_final.to_csv("data/train_data.csv")

In [179]:
#check overlap with external testset
num = []
counter1 = []
counter2 = []
for i in range(len(d_final)):
    for y in range(len(d6)):
        if(d_final["Inchi"][i] == d6["Inchi"][y]):
            num.append(y)
            print(i,":",d_final["Inchi"][i]," // ",y,":",d6["Inchi"][y])
            print(i,":",d_final["Toxicity"][i]," // ",y,":",d6["Toxicity"][y])
            if d_final["Toxicity"][i] != d6["Toxicity"][y]:
                counter1.append(i)
                counter2.append(y)

5300 : InChI=1S/C7H8N4O2/c1-10-5-4(8-3-9-5)6(12)11(2)7(10)13/h3H,1-2H3,(H,8,9)  //  140 : InChI=1S/C7H8N4O2/c1-10-5-4(8-3-9-5)6(12)11(2)7(10)13/h3H,1-2H3,(H,8,9)
5300 : 0  //  140 : 0
6844 : InChI=1S/C4H5NOS/c1-5-4(6)2-3-7-5/h2-3H,1H3  //  36 : InChI=1S/C4H5NOS/c1-5-4(6)2-3-7-5/h2-3H,1H3
6844 : 1  //  36 : 0
6897 : InChI=1S/C11H14O2/c1-3-5-9-6-7-11(13-4-2)10(12)8-9/h3,5-8,12H,4H2,1-2H3  //  473 : InChI=1S/C11H14O2/c1-3-5-9-6-7-11(13-4-2)10(12)8-9/h3,5-8,12H,4H2,1-2H3
6897 : 1  //  473 : 1
6964 : InChI=1S/C22H17ClF3N3O7/c1-33-18(30)21-10-12-9-13(23)3-8-16(12)17(21)27-28(11-35-21)19(31)29(20(32)34-2)14-4-6-15(7-5-14)36-22(24,25)26/h3-9H,10-11H2,1-2H3  //  565 : InChI=1S/C22H17ClF3N3O7/c1-33-18(30)21-10-12-9-13(23)3-8-16(12)17(21)27-28(11-35-21)19(31)29(20(32)34-2)14-4-6-15(7-5-14)36-22(24,25)26/h3-9H,10-11H2,1-2H3
6964 : 1  //  565 : 1
7088 : InChI=1S/C20H16N2O4/c1-2-20(25)14-8-16-17-12(7-11-5-3-4-6-15(11)21-17)9-22(16)18(23)13(14)10-26-19(20)24/h3-8,25H,2,9-10H2,1H3  //  584 : InChI=1S/

7466 : InChI=1S/C19H27N3O4S/c1-21-13-17(16-5-3-4-6-18(16)21)19(23)26-14-15-7-10-22(11-8-15)12-9-20-27(2,24)25/h3-6,13,15,20H,7-12,14H2,1-2H3  //  341 : InChI=1S/C19H27N3O4S/c1-21-13-17(16-5-3-4-6-18(16)21)19(23)26-14-15-7-10-22(11-8-15)12-9-20-27(2,24)25/h3-6,13,15,20H,7-12,14H2,1-2H3
7466 : 0  //  341 : 0
7467 : InChI=1S/C12H18N4O3/c1-7(2)5-16-10-9(11(17)15(3)12(16)18)13-8(14-10)6-19-4/h7H,5-6H2,1-4H3,(H,13,14)  //  353 : InChI=1S/C12H18N4O3/c1-7(2)5-16-10-9(11(17)15(3)12(16)18)13-8(14-10)6-19-4/h7H,5-6H2,1-4H3,(H,13,14)
7467 : 0  //  353 : 0
7468 : InChI=1S/C14H11N/c1-12-6-5-9-14(15-12)11-10-13-7-3-2-4-8-13/h2-9H,1H3  //  360 : InChI=1S/C14H11N/c1-12-6-5-9-14(15-12)11-10-13-7-3-2-4-8-13/h2-9H,1H3
7468 : 0  //  360 : 0
7469 : InChI=1S/C21H22N2O3/c24-21(25)19-12-7-13-23(16-19)14-15-26-22-20(17-8-3-1-4-9-17)18-10-5-2-6-11-18/h1-6,8-12H,7,13-16H2,(H,24,25)  //  363 : InChI=1S/C21H22N2O3/c24-21(25)19-12-7-13-23(16-19)14-15-26-22-20(17-8-3-1-4-9-17)18-10-5-2-6-11-18/h1-6,8-12H,7,13-16H2,(H

In [180]:
len(num),len(counter1),len(counter2)

(92, 2, 2)

In [181]:
d6_ = d6.drop(num)
d6_.reset_index(inplace=True)
d6_.drop("index",axis = 1,inplace = True)
d6_["Toxicity"].value_counts()

Toxicity
0    543
1     71
Name: count, dtype: int64

In [182]:
#Have overlap index, so weird....
#Some conpound's mol will be None when we 
counter1 = set(counter1)
d_final = d_final.drop(counter1)
d_final.reset_index(inplace=True)
d_final.drop("index",axis = 1,inplace = True)
d_final["Mol"] = [Chem.MolFromInchi(x) for x in d_final["Inchi"]]
d_final["Toxicity"].value_counts()

[12:06:10] WARNING: not removing hydrogen atom without neighbors
[12:06:10] Explicit valence for atom # 0 Cl, 5, is greater than permitted
[12:06:10] ERROR: Explicit valence for atom # 0 Cl, 5, is greater than permitted

[12:06:11] WARNING:  Problems/mismatches: Mobile-H( Hydrogens: Locations or number, Number; Charge(s): Do not match; Proton balance: Does not match)

[12:06:11] Explicit valence for atom # 0 Cl, 3, is greater than permitted
[12:06:11] ERROR: Explicit valence for atom # 0 Cl, 3, is greater than permitted

[12:06:12] WARNING: not removing hydrogen atom without neighbors
[12:06:12] WARNING: not removing hydrogen atom without neighbors
[12:06:12] WARNING: not removing hydrogen atom without neighbors
[12:06:12] Explicit valence for atom # 0 Cl, 5, is greater than permitted
[12:06:12] ERROR: Explicit valence for atom # 0 Cl, 5, is greater than permitted

[12:06:12] WARNING:  Problems/mismatches: Mobile-H( Hydrogens: Locations or number; Mobile-H groups: Missing, Attachment p

Toxicity
0    5971
1    1542
Name: count, dtype: int64

In [185]:
for i in range(len(d_final)):
    inchi = Chem.MolFromInchi(d_final["Inchi"][i])
    if inchi == None:
        print(i,":",d_final["Mol"][i],d_final["Toxicity"][i] )
        d_final.drop(index = i,inplace=True,axis=0)
#     else:
#         mol_list.append(All_Mordred_descriptors(inchi))
d_final.reset_index(inplace=True)
d_final.drop("index",axis = 1,inplace=True)

[12:31:38] WARNING: not removing hydrogen atom without neighbors
[12:31:38] Explicit valence for atom # 0 Cl, 5, is greater than permitted
[12:31:38] ERROR: Explicit valence for atom # 0 Cl, 5, is greater than permitted



1322 : None 0


[12:31:39] WARNING:  Problems/mismatches: Mobile-H( Hydrogens: Locations or number, Number; Charge(s): Do not match; Proton balance: Does not match)

[12:31:39] Explicit valence for atom # 0 Cl, 3, is greater than permitted
[12:31:39] ERROR: Explicit valence for atom # 0 Cl, 3, is greater than permitted



5624 : None 0


[12:31:40] WARNING: not removing hydrogen atom without neighbors
[12:31:40] WARNING: not removing hydrogen atom without neighbors
[12:31:40] WARNING: not removing hydrogen atom without neighbors
[12:31:40] Explicit valence for atom # 0 Cl, 5, is greater than permitted
[12:31:40] ERROR: Explicit valence for atom # 0 Cl, 5, is greater than permitted

[12:31:40] WARNING:  Problems/mismatches: Mobile-H( Hydrogens: Locations or number; Mobile-H groups: Missing, Attachment points)



6454 : None 0


[12:31:40] WARNING: not removing hydrogen atom without neighbors
[12:31:40] WARNING: not removing hydrogen atom without neighbors


In [186]:
d_final.to_csv("data/train_raw_data.csv")
d6_.to_csv("data/external_test_raw_data.csv")